# Policy sector


## INDIGO database API endpoint
Setup INDIGO database API endpoint and helper methods for getting individual items from the API. This can be used with the `project`, `fund`, `organisation` and `assessment_resource` endpoints.

In [ ]:
import requests

import plotly.graph_objects as go


INDIGO_DATABASE_API = 'https://golab-indigo-data-store.herokuapp.com/app/api1'


def api_get_item(endpoint, public_id):
    """
    Get individual item details from the API

    E.g. 
    item = api_get_item('project', 'INDIGO-POJ-0158')
    """
    try:
        response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}/{public_id}')
        item = response.json()
        return item
    except Exception as e:
        print(f'\nFailed to retrieve {endpoint} "{public_id}".\nError: {e}')
        return False

## Plot config
Config for adding defined colours to the pie chart.

In [ ]:
COLOUR_MAP_POLICY_SECTOR = {
    'Employment and training': '#ffa600',
    'Child and family welfare': '#ff7c43',
    'Homelessness': '#f95d6a',
    'Health': '#d45087',
    'Education': '#a05195',
    'Criminal justice': '#665191',
    'Agriculture and environment': '#3c77ab',
    'Poverty reduction': '#00629b',
    'Unknown': '#2f4b7c',
}

DEFAULT_PIE_CHART_CONFIG = {
    'hole': 0.35,
    'sort': True,
    'direction': 'clockwise',
    'marker': {
        'line': {
            'color': 'white',
            'width': 2,
        },
    },
    'hovertemplate': '<b>%{label}</b><br>Value: %{value:}<br>Percent: %{percent:.1%}<extra></extra>',
}

## Get project data
Now we actually call the INDIGO API 'project' endpoint and retrieve the data for each of the projects of interest.

In [ ]:
# Call the API and pull down the data for each project
# and store in a dictionary for use later.
public_ids = [
    'INDIGO-POJ-0167', 'INDIGO-POJ-0168', 'INDIGO-POJ-0169', 'INDIGO-POJ-0170', 'INDIGO-POJ-0171', 'INDIGO-POJ-0172',
    'INDIGO-POJ-0173', 'INDIGO-POJ-0174', 'INDIGO-POJ-0175', 'INDIGO-POJ-0176', 'INDIGO-POJ-0177', 'INDIGO-POJ-0178',
    'INDIGO-POJ-0179', 'INDIGO-POJ-0180', 'INDIGO-POJ-0181', 'INDIGO-POJ-0182', 'INDIGO-POJ-0183', 'INDIGO-POJ-0184',
    'INDIGO-POJ-0188', 'INDIGO-POJ-0189', 'INDIGO-POJ-0190', 'INDIGO-POJ-0192', 'INDIGO-POJ-0193', 'INDIGO-POJ-0194',
    'INDIGO-POJ-0195', 'INDIGO-POJ-0198', 'INDIGO-POJ-0199', 'INDIGO-POJ-0200', 'INDIGO-POJ-0201',
]

data = {}
endpoint = 'project'

for public_id in public_ids:
    data[public_id] = api_get_item(endpoint, public_id)

## Generate plot data


In [ ]:
policy_sectors = {}

for public_id in public_ids:
    project_data = data[public_id]['project']['data']

    purpose_and_classifications = project_data.get('purpose_and_classifications') or {}
    policy_sector = purpose_and_classifications.get('policy_sector') or {}
    value = policy_sector.get('value') or 'Unknown'
    
    if value in policy_sectors:
        policy_sectors[value] += 1
    else:
        policy_sectors[value] = 1

colours = [COLOUR_MAP_POLICY_SECTOR.get(label, '#00629b') for label in policy_sectors.keys()]

print(policy_sectors)

plot_data = [
    go.Pie(
        name='Policy Sectors',
        labels=list(policy_sectors.keys()),
        values=list(policy_sectors.values()),
        marker_colors=colours,
        **DEFAULT_PIE_CHART_CONFIG,
    )
]

## Build the figure

In [ ]:
fig = go.Figure(data=plot_data)

fig.update_layout(
    height=600,
    title='Policy sector example',
    showlegend=True,
)

fig.show()

## Important Notice and Disclaimer on INDIGO Data
<sub><sup>
INDIGO data are shared for research and policy analysis purposes. INDIGO data can be used to support a range of insights, for example, to understand the social outcomes that projects aim to improve, the network of organisations across projects, trends, scales, timelines and summary information. The collaborative system by which we collect, process, and share data is designed to advance data-sharing norms, harmonise data definitions and improve data use. These data are NOT shared for auditing, investment, or legal purposes. Please independently verify any data that you might use in decision making. We provide no guarantees or assurances as to the quality of these data. Data may be inaccurate, incomplete, inconsistent, and/or not current for various reasons: INDIGO is a collaborative and iterative initiative that mostly relies on projects all over the world volunteering to share their data. We have a system for processing information and try to attribute data to named sources, but we do not audit, cross-check, or verify all information provided to us. It takes time and resources to share data, which may not have been included in a project’s budget. Many of the projects are ongoing and timely updates may not be available. Different people may have different interpretations of data items and definitions. Even when data are high quality, interpretation or generalisation to different contexts may not be possible and/or requires additional information and/or expertise. Help us improve our data quality: email us at indigo@bsg.ox.ac.uk if you have data on new projects, changes or performance updates on current projects, clarifications or corrections on our data, and/or confidentiality or sensitivity notices. Please also give input via the INDIGO Data Definitions Improvement Tool and INDIGO Feedback Questionnaire.
</sup></sub>
